# Tema 7: Funciones de orden superior
[José A. Alonso](https://www.cs.us.es/~jalonso) (Univ. de Sevilla, 3 de agosto de 2019)

> __Notas:__ 
+ La versión interactiva de este tema se encuentra en [Binder](https://mybinder.org/v2/gh/jaalonso/Temas_interactivos_de_PF_con_Haskell/master?urlpath=lab/tree/temas/Tema-07.ipynb).
+ Se desactiva el [corrector estilo de Haskell](https://github.com/gibiansky/IHaskell/wiki#opt-no-lint).

In [1]:
:opt no-lint

**Librerías auxiliares**

+ En este tema se usarán las siguientes librerías:

In [2]:
import Data.Char
import Test.QuickCheck
import Test.QuickCheck.Function

Funciones de orden superior
===========================

**Funciones de orden superior**

+ Una función es *de orden superior* si toma una función como argumento o
  devuelve una función como resultado.

+ `(dosVeces f x)` es el resultado de aplicar `f` a `f x`. Por ejemplo,

```sesion
dosVeces (*3) 2           ==  18
dosVeces reverse [2,5,7]  ==  [2,5,7]  
```

In [3]:
dosVeces :: (a -> a) -> a -> a
dosVeces f x = f (f x)

In [4]:
dosVeces (*3) 2           

18

In [5]:
dosVeces reverse [2,5,7]  

[2,5,7]

+ Prop: `dosVeces reverse = id`, donde `id` es la función identidad.

```haskell
id :: a -> a
id x =  x
```

**Usos de las funciones de orden superior**

+ Definición de patrones de programación.
    + Aplicación de una función a todos los elementos de una lista.
    + Filtrado de listas por propiedades.
    + Patrones de recursión sobre listas.

+ Diseño de lenguajes de dominio específico:
    + Lenguajes para procesamiento de mensajes.
    + Analizadores sintácticos.
    + Procedimientos de entrada/salida.

+ Uso de las propiedades algebraicas de las funciones de orden superior para
  razonar sobre programas.

Procesamiento de listas
=======================

La función `map`
----------------

**La función `map`: Definición**

+ `(map f xs)` es la lista obtenida aplicando `f` a cada elemento de `xs`. Por
    ejemplo,

In [6]:
map (*2) [3,4,7]     

[6,8,14]

In [7]:
map sqrt [1,2,4]     

[1.0,1.4142135623730951,2.0]

In [8]:
map even [1..5]      

[False,True,False,True,False]

+ Definición de `map` por comprensión:

In [9]:
map' :: (a -> b) -> [a] -> [b]
map' f xs =  [f x | x <- xs]

In [10]:
map' (*2) [3,4,7]

[6,8,14]

+ Definición de `map` por recursión:

In [11]:
map'' :: (a -> b) -> [a] -> [b]
map'' _ []     = []
map'' f (x:xs) = f x : map'' f xs

In [12]:
map' (*2) [3,4,7]

[6,8,14]

**Relación entre `sum` y `map`**

+ La función `sum`:

In [13]:
sum :: [Int] -> Int
sum []     = 0                
sum (x:xs) = x + sum xs       

In [14]:
sum [3,2,5]

10

+ Propiedad: `sum (map (2*) xs) = 2 * sum xs`

+ Comprobación con QuickCheck:

In [15]:
prop_sum_map :: [Int] -> Bool
prop_sum_map xs = sum (map (2*) xs) == 2 * sum xs

In [16]:
quickCheck prop_sum_map

+++ OK, passed 100 tests.

La función `filter`
-------------------

**La función `filter`**

+ `filter p xs` es la lista de los elementos de `xs` que cumplen la propiedad
  `p`. Por ejemplo,

```sesion
filter even [1,3,5,4,2,6,1] == [4,2,6]
filter (>3) [1,3,5,4,2,6,1] == [5,4,6]  
```

+ Definición de `filter` por comprensión:

In [17]:
filter :: (a -> Bool) -> [a] -> [a]
filter p xs =  [x | x <- xs, p x]

In [18]:
filter even [1,3,5,4,2,6,1]

[4,2,6]

In [19]:
filter even [1,3,5,4,2,6,1]

[4,2,6]

+ Definición de `filter` por recursión:

In [20]:
filter :: (a -> Bool) -> [a] -> [a]
filter _ []                 = []
filter p (x:xs) | p x       = x : filter p xs
                | otherwise = filter p xs

In [21]:
filter even [1,3,5,4,2,6,1]

[4,2,6]

In [22]:
filter even [1,3,5,4,2,6,1]

[4,2,6]

**Uso conjunto de `map` y `filter`**

+ `sumaCuadradosPares xs` es la suma de los cuadrados de los números pares de
  la lista `xs`. Por ejemplo,

```sesion
sumaCuadradosPares [1..5]  ==  20  
```

In [23]:
sumaCuadradosPares :: [Int] -> Int
sumaCuadradosPares xs = sum (map (^2) (filter even xs))

In [24]:
sumaCuadradosPares [1..5]

20

+ Definición por comprensión:

In [25]:
sumaCuadradosPares' :: [Int] -> Int
sumaCuadradosPares' xs = sum [x^2 | x <- xs, even x]  

In [26]:
sumaCuadradosPares [1..5]

20

**Predefinidas de orden superior para procesar listas**

+ `all p xs` se verifica si todos los elementos de `xs` cumplen la propiedad
  `p`. Por ejemplo,

In [27]:
all odd [1,3,5]  

True

In [28]:
all odd [1,3,6]  

False

+ `any p xs` se verifica si algún elemento de `xs` cumple la propiedad `p`. Por
  ejemplo,

In [29]:
any odd [1,3,5]  

True

In [30]:
any odd [2,4,6]  

False

+ `takeWhile p xs` es la lista de los elementos iniciales de `xs` que verifican
  el predicado `p`. Por ejemplo,

In [31]:
takeWhile even [2,4,6,7,8,9]

[2,4,6]

+ `dropWhile p xs` es la lista `xs` sin los elementos iniciales que verifican
  el predicado `p`. Por ejemplo,

In [32]:
dropWhile even [2,4,6,7,8,9] 

[7,8,9]

Función de plegado por la derecha: `foldr`
==========================================

**Esquema básico de recursión sobre listas**

+ Ejemplos de definiciones recursivas:

In [33]:
sum []         = 0
sum (x:xs)     = x + sum xs

product []     = 1
product (x:xs) = x * product xs

or []          = False
or (x:xs)      = x || or xs

and []         = True
and (x:xs)     = x && and xs

+ Esquema básico de recursión sobre listas:
```haskell
f []     = v
f (x:xs) = x `op` (f xs)
```

**El patrón `foldr`**

+ Redefiniciones con el patrón `foldr`

In [34]:
sum     = foldr (+) 0
product = foldr (*) 1
or      = foldr (||) False
and     = foldr (&&) True

+ Definición del patrón `foldr`
```haskell
foldr :: (a -> b -> b) -> b -> [a] -> b
foldr f v []     =  v
foldr f v (x:xs) =  f x (foldr f v xs)
```

**Visión no recursiva de `foldr`**

+ Cálculo con `sum`:

```sesion
sum [2,3,5]
= foldr (+) 0 [2,3,5]       [def. de sum] 
= foldr (+) 0 2:(3:(5:[]))  [notación de lista] 
=             2+(3+(5+0))   [sustituir (:) por (+) y 
                                       []  por 0] 
= 10`                       [aritmética]
```

+ Cálculo con `sum`:

```sesion
product [2,3,5]  
= foldr (*) 1 [2,3,5]      [def. de sum] 
= foldr (*) 1 2:(3:(5:[])) [notación de lista] 
=             2*(3*(5*1))  [sustituir (:) por (*) y 
                                      []  por 1] 
= 30                       [aritmética]
```

+ Cálculo de `foldr f v xs` 
    + Sustituir en `xs` los `(:)` por `f` y `[]` por `v`.

**Definición de la longitud mediante `foldr`**

+ Ejemplo de cálculo de la longitud:

```sesion
longitud [2,3,5]
= longitud 2:(3:(5:[]))
=          1+(1+(1+0))      [Sustituciones]
= 3  
```

+ Sustituciones:
    + los `(:)` por `(\x y -> 1+y)`
    + la `[]` por `0`

+ Definición de `length` usando `foldr`

In [35]:
longitud :: [a] -> Int
longitud = foldr (\x y -> 1+y) 0

In [36]:
longitud [4,2,5]

3

**Definición de la inversa mediante `foldr`**

+ Ejemplo de cálculo de la inversa:

```sesion
inversa [2,3,5]
= inversa 2:(3:(5:[]))
=         (([] ++ [5]) ++ [3]) ++ [2]  [Sustituciones]
= [5,3,2]
```

+ Sustituciones:
    + los `(:)` por `(\x y -> y ++ [x])`
    + la `[]` por `[]`

+ Definición de `inversa` usando `foldr`

In [37]:
inversa :: [a] -> [a]
inversa = foldr (\x y -> y ++ [x]) []

In [38]:
inversa [3,2,6]

[6,2,3]

**Definición de la concatenación mediante `foldr`**

+ Ejemplo de cálculo de la concatenación:

```sesion
conc [2,3,5] [7,9]
= conc 2:(3:(5:[])) [7,9]
=      2:(3:(5:[7,9]))       [Sustituciones]
= [2,3,5,7,9]
```

+ Sustituciones:
    + los `(:)` por `(:)`
    + la `[]` por `ys`

+ Definición de `conc` usando `foldr`

In [39]:
conc xs ys = (foldr (:) ys) xs

In [40]:
conc [3,2,5] [4,9,7]

[3,2,5,4,9,7]

Función de plegado por la izquierda: `foldl`
============================================

**Definición de suma de lista con acumuladores**

+ Definición de `suma` con acumuladores:

In [41]:
suma :: [Integer] -> Integer
suma = sumaAux 0
    where sumaAux v []     = v
          sumaAux v (x:xs) = sumaAux (v+x) xs

In [42]:
suma [2,3,7]

12

+ Cálculo con `suma`:

```sesion
suma [2,3,7] 
= sumaAux 0 [2,3,7]  
= sumaAux (0+2) [3,7]  
= sumaAux 2 [3,7]  
= sumaAux (2+3) [7]  
= sumaAux 5 [7]  
= sumaAux (5+7) []  
= sumaAux 12 []
= 12  
```

**Patrón de definición de recursión con acumulador**

+ Patrón de definición (generalización de `sumaAux`):

```haskell
f v []     = v
f v (x:xs) = f (v*x) xs
```

+ Definición con el patrón `foldl`:

In [43]:
suma    = foldl (+) 0
product = foldl (*) 1
or      = foldl (||) False
and     = foldl (&&) True

**Definición de `foldl`**

+ Definición de `foldl`:

In [44]:
foldl :: (a -> b -> a) -> a -> [b ] -> a
foldl f v []     =  v
foldl f v (x:xs) =  foldl f (f v x ) xs

+ Diferencia entre `foldr` y `foldl`:

```sesion
(foldr (-) 0) [3,4,2] =     3-(4-(2-0)) = 1
(foldl (-) 0) [3,4,2] = ((0-3)-4)-2     = -9
```

Composición de funciones
========================

**Composición de funciones**

+ Definición de la composición de dos funciones:


In [45]:
(.) :: (b -> c) -> (a -> b) -> a -> c
f . g  = \x -> f (g x)

**Uso de composición para simplificar definiciones**

+ Definiciones sin composición:

In [46]:
doVeces f x           = f (f x )

sumaCuadradosPares ns = sum (map (^2) (filter even ns))

+ Definiciones con composición:

In [47]:
dosVeces f         = f . f

sumaCuadradosPares = sum . map (^2) . filter even

**Composición de una lista de funciones**

+ La función identidad:

In [48]:
id :: a -> a
id =  \x -> x

+ `(composicionLista fs)` es la composición de la lista de funciones `fs`. Por
  ejemplo,

```sesion
composicionLista [(*2),(^2)] 3       ==  18
composicionLista [(^2),(*2)] 3       ==  36
composicionLista [(/9),(^2),(*2)] 3  ==  4.0
```

In [49]:
composicionLista :: [a -> a] -> (a -> a)
composicionLista =  foldr (.) id

In [50]:
composicionLista [(*2),(^2)] 3       

18

In [51]:
composicionLista [(^2),(*2)] 3       

36

In [52]:
composicionLista [(/9),(^2),(*2)] 3  

4.0

Caso de estudio: Codificación binaria y transmisión de cadenas
==============================================================

+ Objetivos:
    + Definir una función que convierta una cadena en una lista de ceros
      y unos junto con otra función que realice la conversión opuesta. 
    + Simular la transmisión de cadenas mediante ceros y unos.

+ Los números binarios se representan mediante listas de bits en orden
  inverso. Un bit es cero o uno. Por ejemplo, el número 1101 se representa
  por [1,0,1,1]. 

+ El tipo `Bit` es el de los bits.

In [53]:
type Bit = Int

Cambio de bases
---------------

**Cambio de bases: De binario a decimal**

+ `(bin2int x)` es el número decimal correspondiente al número binario `x`. Por
  ejemplo,

```sesion
bin2int [1,0,1,1]  ==  13
```

+ El cálculo es

```sesion
bin2int [1,0,1,1]
= bin2int 1:(0:(1:(1:[])))
=         1+2*(0+2*(1+2*(1+2*0)))
= 13
```

In [54]:
bin2int :: [Bit] -> Int
bin2int =  foldr (\x y -> x + 2*y) 0

In [55]:
bin2int [1,0,1,1]

13

**Cambio de base: De decimal a binario**

+ `(int2bin x)` es el número binario correspondiente al número decimal `x`. Por
  ejemplo,

```sesion
int2bin 13  ==  [1,0,1,1]  
```

In [56]:
int2bin :: Int -> [Bit]
int2bin n | n < 2     = [n]
          | otherwise = n `mod` 2 : int2bin (n `div` 2)

In [57]:
int2bin 13

[1,0,1,1]

+ El cálculo es

```sesion
int2bin 13 
= 13 `mod` 2 : int2bin (13 `div` 2)  
= 1 : int2bin (6 `div` 2)  
= 1 : (6 `mod` 2 : int2bin (6 `div` 2))
= 1 : (0 : int2bin 3)
= 1 : (0 : (3 `mod` 2 : int2bin (3 `div` 2)))
= 1 : (0 : (1 : int2bin 1))
= 1 : (0 : (1 : (1 : int2bin 0)))
= 1 : (0 : (1 : (1 : [])))
= [1,0,1,1]
```

**Cambio de base: Comprobación de propiedades**

+ Propiedad: Al pasar un número natural a binario con `int2bin` y el resultado
  a decimal con `bin2int` se obtiene el número inicial.

In [58]:
prop_int_bin :: Int -> Bool
prop_int_bin x =
    bin2int (int2bin y) == y
    where y = abs x

+ Comprobación:

In [59]:
quickCheck prop_int_bin

+++ OK, passed 100 tests.

Codificación y descodificación
------------------------------

**Creación de octetos**

+ Un octeto es un grupo de ocho bits.

+ `(creaOcteto bs)` es el octeto correspondiente a la lista de bits `bs`; es
  decir, los 8 primeros elementos de `bs` si su longitud es mayor o igual que
  8 y la lista de 8 elemento añadiendo ceros al final de `bs` en caso
  contrario. Por ejemplo,

```sesion
ghci> creaOcteto [1,0,1,1,0,0,1,1,1,0,0,0]
[1,0,1,1,0,0,1,1]
ghci> creaOcteto [1,0,1,1]             
[1,0,1,1,0,0,0,0]
```

In [60]:
creaOcteto :: [Bit] -> [Bit]
creaOcteto bs =  take 8 (bs ++ repeat 0)

In [61]:
creaOcteto [1,0,1,1,0,0,1,1,1,0,0,0]

[1,0,1,1,0,0,1,1]

In [62]:
creaOcteto [1,0,1,1]             

[1,0,1,1,0,0,0,0]

+ `(repeat x)` es una lista infinita cuyo único elemento es `x`. Por ejemplo,

In [63]:
take 3 (repeat 5) == [5,5,5]

True

**Codificación**

+ `(codifica c)` es la codificación de la cadena `c` como una lista de bits
  obtenida convirtiendo cada carácter en un número Unicode, convirtiendo cada
  uno de dichos números en un octeto y concatenando los octetos para obtener
  una lista de bits. Por ejemplo,

```sesion
ghci> codifica "abc"
[1,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,1,1,0,0,0,1,1,0]
```

In [64]:
codifica :: String -> [Bit]
codifica =  concat . map (creaOcteto . int2bin . ord)

In [65]:
codifica "abc"

[1,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,1,1,0,0,0,1,1,0]

+ `(concat xss)` es la lista obtenida concatenando la lista de listas
  `xss`. Por ejemplo,

In [66]:
concat [[1,5],[2],[4,5,3]]

[1,5,2,4,5,3]

**Codificación**

+ Ejemplo de codificación,

```sesion
codifica "abc" 
= concat . map (creaOcteto . int2bin . ord) "abc"
= concat . map (creaOcteto . int2bin . ord) ['a','b','c']
= concat [creaOcteto . int2bin . ord 'a', 
          creaOcteto . int2bin . ord 'b', 
          creaOcteto . int2bin . ord 'c']
= concat [creaOcteto [1,0,0,0,0,1,1], 
          creaOcteto [0,1,0,0,0,1,1], 
          creaOcteto [1,1,0,0,0,1,1]]
= concat [[1,0,0,0,0,1,1,0], 
          [0,1,0,0,0,1,1,0], 
          [1,1,0,0,0,1,1,0]]
= [1,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,1,1,0,0,0,1,1,0]
```

**Separación de octetos**

+ `(separaOctetos bs)` es la lista obtenida separando la lista de bits `bs` en
  listas de 8 elementos. Por ejemplo,

```sesion
ghci> separaOctetos [1,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0]
[[1,0,0,0,0,1,1,0],[0,1,0,0,0,1,1,0]]
```

In [67]:
separaOctetos :: [Bit] -> [[Bit]]
separaOctetos [] = []
separaOctetos bs =  
    take 8 bs : separaOctetos (drop 8 bs)

In [68]:
separaOctetos [1,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0]

[[1,0,0,0,0,1,1,0],[0,1,0,0,0,1,1,0]]

**Descodificación**

+ `(descodifica bs)` es la cadena correspondiente a la lista de bits
  `bs`. Por ejemplo, 

```sesion
ghci> descodifica [1,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,1,1,0,0,0,1,1,0]
"abc"
```

In [69]:
descodifica :: [Bit] -> String
descodifica =  map (chr . bin2int) . separaOctetos

In [70]:
descodifica [1,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,1,1,0,0,0,1,1,0]

"abc"

+ Ejemplo de cálculo:

```sesion
descodifica [1,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,1,1,0,0,0,1,1,0]
= (map (chr . bin2int) . separaOctetos) 
  [1,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,1,1,0,0,0,1,1,0]
= map (chr . bin2int) [[1,0,0,0,0,1,1,0],[0,1,0,0,0,1,1,0],[1,1,0,0,0,1,1,0]]
= [(chr . bin2int) [1,0,0,0,0,1,1,0],
   (chr . bin2int) [0,1,0,0,0,1,1,0],
   (chr . bin2int) [1,1,0,0,0,1,1,0]]
= [chr 97, chr 98, chr 99]
= "abc"
```

**Transmisión**

+ Los canales de transmisión pueden representarse mediante funciones que
  transforman cadenas de bits en cadenas de bits.

+ `(transmite c t)` es la cadena obtenida transmitiendo la cadena `t` a través
  del canal `c`. Por ejemplo,

```sesion
ghci> transmite id "Texto por canal correcto"
"Texto por canal correcto"
```

In [71]:
transmite :: ([Bit] -> [Bit]) -> String -> String
transmite canal =  descodifica . canal . codifica

In [72]:
transmite id "Texto por canal correcto"

"Texto por canal correcto"

**Corrección de la transmisión**

+ Propiedad: Al trasmitir cualquier cadena por el canal identidad se obtiene la
  cadena.

In [73]:
prop_transmite :: String -> Bool
prop_transmite cs =
    transmite id cs' == cs'
    where cs'= [c | c <- cs, c `elem` ['a'..'z'] ++ ['A'..'Z']]

+ Comprobación de la corrección:

In [74]:
quickCheck prop_transmite

+++ OK, passed 100 tests.

Bibliografía
============

+ R. Bird. *Introducción a la programación funcional con Haskell*. Prentice
  Hall, 2000.
    + Cap. 4: Listas.

+ G. Hutton. *Programming in Haskell*. Cambridge University Press, 2007.
    + Cap. 7: Higher-order functions.

+ B.C. Ruiz, F. Gutiérrez, P. Guerrero y J.E. Gallardo. *Razonando con
  Haskell*. Thompson, 2004.
    + Cap. 8: Funciones de orden superior y polimorfismo.

+ S. Thompson. *Haskell: The Craft of Functional Programming*, Second
  Edition. Addison-Wesley, 1999. 
    + Cap. 9: Generalization: patterns of computation.
    + Cap. 10: Functions as values.